In [2]:
import pickle

with open('dataframe.pkl', 'rb') as file:
    df = pickle.load(file)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4646773 entries, 0 to 4646773
Data columns (total 27 columns):
 #   Column           Dtype         
---  ------           -----         
 0   FlagImpaye       category      
 1   Montant          float64       
 2   DateTransaction  datetime64[ns]
 3   CodeDecision     object        
 4   VerifianceCPT1   int32         
 5   VerifianceCPT2   int32         
 6   VerifianceCPT3   int32         
 7   D2CB             int32         
 8   ScoringFP1       float64       
 9   ScoringFP2       float64       
 10  ScoringFP3       float64       
 11  TauxImpNb_RB     float64       
 12  TauxImpNB_CPM    float64       
 13  EcartNumCheq     int32         
 14  NbrMagasin3J     int32         
 15  DiffDateTr1      float64       
 16  DiffDateTr2      float64       
 17  DiffDateTr3      float64       
 18  CA3TRetMtt       float64       
 19  CA3TR            float64       
 20  Heure            int32         
 21  Year             int32         
 22 

### On définit la partie Train et Test

In [4]:
# Définition des dates limites pour les ensembles d'apprentissage et de test
train_inf = '2017-02-01'
train_sup = '2017-08-31'
test_inf = '2017-09-01'
test_sup = '2017-11-30'

In [5]:
train = df.loc[(df['DateTransaction'] >= train_inf) & (df['DateTransaction'] <= train_sup)]
X_train = train.drop(columns=['FlagImpaye','CodeDecision','DateTransaction'])
y_train = train['FlagImpaye']

In [6]:
test = df.loc[(df['DateTransaction'] >= test_inf) & (df['DateTransaction'] <= test_sup)]
X_test = test.drop(columns=['FlagImpaye','CodeDecision','DateTransaction'])
y_test = test['FlagImpaye']

In [7]:
len(X_train)

3888468

In [8]:
len(X_test)

737068

## Test de chaque méthode d'échantillonage par modeles

In [9]:
from sklearn import metrics
from sklearn.metrics import f1_score

In [10]:
!pip install 'imblearn'
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN, BorderlineSMOTE

ERROR: Invalid requirement: "'imblearn'"

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
!pip install xgboost


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


On fait tourner DecisionTreeClassifier, RandomForestClassifier, GradientBoostingClassifier, SVC, adaboost, xgboost et LogisticRegression:

In [12]:
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from imblearn.over_sampling import RandomOverSampler, SMOTE, BorderlineSMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from tqdm import tqdm

In [12]:
# On définit les stratégies d'échantillonnage
sampling_strategies = [0.05, 0.1]  

#On garde deux oversampling et deux undersampling
samplers = [RandomOverSampler, SMOTE, RandomUnderSampler, NearMiss]

# Précalcule les échantillons resamplés pour chaque méthode d'échantillonnage
resampled_data = {}
for sampler in tqdm(samplers, desc="Resampling Progress"):
    for strategy in tqdm(sampling_strategies, desc=f"{sampler.__name__} Progress", leave=False):
        key = (sampler, strategy)
        X_resampled, y_resampled = sampler(sampling_strategy=strategy).fit_resample(X_train, y_train)
        resampled_data[key] = (X_resampled, y_resampled)

Resampling Progress: 100%|██████████| 4/4 [02:52<00:00, 43.12s/it]


Modèle DecisionTreeClassifier:

In [29]:
# On définit le modèle à tester
models = [
    DecisionTreeClassifier()
]

# Initialisation d'un DataFrame pour stocker les résultats
results_df_1 = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# On parcourt chaque modèle
for model in models:
    # On parcourt chaque échantillonneur
    for sampler in samplers:
        # On parcourt chaque valeur de sampling_strategy
        for strategy in sampling_strategies:
            # Récupére les échantillons resamplés précalculés
            X_resampled, y_resampled = resampled_data[(sampler, strategy)]

            # Initialise et entraîne le modèle
            model.fit(X_resampled, y_resampled)

            # Faire des prédictions sur l'ensemble de test
            y_pred = model.predict(X_test)

            # Évalue les performances du modèle
            f1 = f1_score(y_test, y_pred)

            # Ajoute les résultats au DataFrame
            results_df_1 = pd.concat([results_df_1, pd.DataFrame({
                'Model': [model.__class__.__name__],
                'Sampler': [sampler.__name__],
                'Sampling Strategy': [strategy],
                'F1 Score': [f1]
            })], ignore_index=True)

            # Affiche le message de progression
            tqdm.write(f"Model: {model.__class__.__name__}, Sampler: {sampler.__name__}, Strategy: {strategy}, F1 Score: {f1}")

# Affiche le tableau récapitulatif
print(results_df_1)
results_df_1.to_pickle('dataframe_recap_sampling_1.pkl')

C:\Users\marevel\AppData\Local\Temp\ipykernel_14404\260053566.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df_1 = pd.concat([results_df_1, pd.DataFrame({


Model: DecisionTreeClassifier, Sampler: RandomOverSampler, Strategy: 0.05, F1 Score: 0.030195815287012748
Model: DecisionTreeClassifier, Sampler: RandomOverSampler, Strategy: 0.1, F1 Score: 0.026262189853104554
Model: DecisionTreeClassifier, Sampler: SMOTE, Strategy: 0.05, F1 Score: 0.03714936856354632
Model: DecisionTreeClassifier, Sampler: SMOTE, Strategy: 0.1, F1 Score: 0.03639382151402204
Model: DecisionTreeClassifier, Sampler: RandomUnderSampler, Strategy: 0.05, F1 Score: 0.03486636000379733
Model: DecisionTreeClassifier, Sampler: RandomUnderSampler, Strategy: 0.1, F1 Score: 0.03289138545840045
Model: DecisionTreeClassifier, Sampler: NearMiss, Strategy: 0.05, F1 Score: 0.02929911735260891
Model: DecisionTreeClassifier, Sampler: NearMiss, Strategy: 0.1, F1 Score: 0.025979530864366336
                    Model             Sampler  Sampling Strategy  F1 Score
0  DecisionTreeClassifier   RandomOverSampler               0.05  0.030196
1  DecisionTreeClassifier   RandomOverSampler      

Modèle RandomForestClassifier:

In [34]:

# On définit le modèle à tester
models = [
    RandomForestClassifier()
]

# Initialisation d'un DataFrame pour stocker les résultats
results_df_2 = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# On parcourt chaque modèle
for model in models:
    # On parcourt chaque échantillonneur
    for sampler in samplers:
        # On parcourt chaque valeur de sampling_strategy
        for strategy in sampling_strategies:
            # Récupére les échantillons resamplés précalculés
            X_resampled, y_resampled = resampled_data[(sampler, strategy)]

            # Initialise et entraîne le modèle
            model.fit(X_resampled, y_resampled)

            # Fait des prédictions sur l'ensemble de test
            y_pred = model.predict(X_test)

            # Évalue les performances du modèle
            f1 = f1_score(y_test, y_pred)

            # Ajoute les résultats au DataFrame
            results_df_2 = pd.concat([results_df_2, pd.DataFrame({
                'Model': [model.__class__.__name__],
                'Sampler': [sampler.__name__],
                'Sampling Strategy': [strategy],
                'F1 Score': [f1]
            })], ignore_index=True)

            # Affiche le message de progression
            tqdm.write(f"Model: {model.__class__.__name__}, Sampler: {sampler.__name__}, Strategy: {strategy}, F1 Score: {f1}")

# Affiche le tableau récapitulatif
print(results_df_2)
results_df_2.to_pickle('dataframe_recap_sampling_2.pkl')

Model: RandomForestClassifier, Sampler: NearMiss, Strategy: 0.1, F1 Score: 0.02952237759137063
                    Model   Sampler  Sampling Strategy  F1 Score
0  RandomForestClassifier  NearMiss                0.1  0.029522


C:\Users\marevel\AppData\Local\Temp\ipykernel_14404\4242767768.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df_2_bis = pd.concat([results_df_2_bis, pd.DataFrame({


Modèle SVC:

In [13]:
# On définit le modèle à tester
models = [
    SVC()
]

# Initialisation d'un DataFrame pour stocker les résultats
results_df_3 = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# On parcourt chaque modèle
for model in models:
    # On parcourt chaque échantillonneur
    for sampler in samplers:
        # On parcourt chaque valeur de sampling_strategy
        for strategy in sampling_strategies:
            # Récupére les échantillons resamplés précalculés
            X_resampled, y_resampled = resampled_data[(sampler, strategy)]

            # Initialise et entraîne le modèle
            model.fit(X_resampled, y_resampled)

            # Fait des prédictions sur l'ensemble de test
            y_pred = model.predict(X_test)

            # Évalue les performances du modèle
            f1 = f1_score(y_test, y_pred)

            # Ajoute les résultats au DataFrame
            results_df_3 = pd.concat([results_df_3, pd.DataFrame({
                'Model': [model.__class__.__name__],
                
                'Sampler': [sampler.__name__],
                'Sampling Strategy': [strategy],
                'F1 Score': [f1]
            })], ignore_index=True)

            # Affiche le message de progression
            tqdm.write(f"Model: {model.__class__.__name__}, Sampler: {sampler.__name__}, Strategy: {strategy}, F1 Score: {f1}")

# Affiche le tableau récapitulatif
print(results_df_3)
results_df_3.to_pickle('dataframe_recap_sampling_3.pkl')

Modèle AdaBoostClassifier:

In [ ]:
# On définit le modèle à tester
models = [
    AdaBoostClassifier()
]

# Initialisation d'un DataFrame pour stocker les résultats
results_df_4 = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# On parcourt chaque modèle
for model in models:
    # On parcourt chaque échantillonneur
    for sampler in samplers:
        # On parcourt chaque valeur de sampling_strategy
        for strategy in sampling_strategies:
            # Récupére les échantillons resamplés précalculés
            X_resampled, y_resampled = resampled_data[(sampler, strategy)]

            # Initialise et entraîne le modèle
            model.fit(X_resampled, y_resampled)

            # Fait des prédictions sur l'ensemble de test
            y_pred = model.predict(X_test)

            # Évalue les performances du modèle
            f1 = f1_score(y_test, y_pred)

            # Ajoute les résultats au DataFrame
            results_df_4 = pd.concat([results_df_4, pd.DataFrame({
                'Model': [model.__class__.__name__],
                'Sampler': [sampler.__name__],
                'Sampling Strategy': [strategy],
                'F1 Score': [f1]
            })], ignore_index=True)

            # Affiche le message de progression
            tqdm.write(f"Model: {model.__class__.__name__}, Sampler: {sampler.__name__}, Strategy: {strategy}, F1 Score: {f1}")

# Affiche le tableau récapitulatif
print(results_df_4)
results_df_4.to_pickle('dataframe_recap_sampling_4.pkl')

Modèle XGBClassifier:

In [ ]:
# On définit le modèle à tester
models = [
    XGBClassifier()
]

# Initialisation d'un DataFrame pour stocker les résultats
results_df_5 = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# On parcourt chaque modèle
for model in models:
    # On parcourt chaque échantillonneur
    for sampler in samplers:
        # On parcourt chaque valeur de sampling_strategy
        for strategy in sampling_strategies:
            # Récupére les échantillons resamplés précalculés
            X_resampled, y_resampled = resampled_data[(sampler, strategy)]

            # Initialise et entraîne le modèle
            model.fit(X_resampled, y_resampled)

            # Fait des prédictions sur l'ensemble de test
            y_pred = model.predict(X_test)

            # Évalue les performances du modèle
            f1 = f1_score(y_test, y_pred)

            # Ajoute les résultats au DataFrame
            results_df_5 = pd.concat([results_df_5, pd.DataFrame({
                'Model': [model.__class__.__name__],
                'Sampler': [sampler.__name__],
                'Sampling Strategy': [strategy],
                'F1 Score': [f1]
            })], ignore_index=True)

            # Affiche le message de progression
            tqdm.write(f"Model: {model.__class__.__name__}, Sampler: {sampler.__name__}, Strategy: {strategy}, F1 Score: {f1}")

# Affiche le tableau récapitulatif
print(results_df_5)
results_df_5.to_pickle('dataframe_recap_sampling_5.pkl')

Modèle LogisticRegression:

In [ ]:
from sklearn.linear_model import LogisticRegression

# On définit le modèle à tester
models = [
    LogisticRegression(max_iter= 1000) 
]

# Initialisation d'un DataFrame pour stocker les résultats
results_df_6 = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# On parcourt chaque modèle
for model in models:
    # On parcourt chaque échantillonneur
    for sampler in samplers:
        # On parcourt chaque valeur de sampling_strategy
        for strategy in sampling_strategies:
            # Récupére les échantillons resamplés précalculés
            X_resampled, y_resampled = resampled_data[(sampler, strategy)]

            # Initialise et entraîne le modèle de régression logistique
            model.fit(X_resampled, y_resampled)

            # Fait des prédictions sur l'ensemble de test
            y_pred = model.predict(X_test)

            # Évalue les performances du modèle
            f1 = f1_score(y_test, y_pred)

            # Ajoute les résultats au DataFrame
            results_df_6 = pd.concat([results_df_6, pd.DataFrame({
                'Model': [model.__class__.__name__],
                'Sampler': [sampler.__name__],
                'Sampling Strategy': [strategy],
                'F1 Score': [f1]
            })], ignore_index=True)

            # Affiche le message de progression
            tqdm.write(f"Model: {model.__class__.__name__}, Sampler: {sampler.__name__}, Strategy: {strategy}, F1 Score: {f1}")

# Affiche le tableau récapitulatif
print(results_df_6)
results_df_6.to_pickle('dataframe_recap_sampling_6_1000.pkl')


On réunit les résultats dans un seul tableau recapitulatif

In [3]:
results_df_1 = pd.read_pickle('dataframe_recap_sampling_1.pkl')
results_df_2 = pd.read_pickle('dataframe_recap_sampling_2.pkl')
results_df_4 = pd.read_pickle('dataframe_recap_sampling_4.pkl')
results_df_5 = pd.read_pickle('dataframe_recap_sampling_5.pkl')
results_df_6 = pd.read_pickle('dataframe_recap_sampling_6_1000.pkl')

In [5]:
results_df = pd.concat([results_df_1, results_df_2, results_df_4, results_df_5, results_df_6], ignore_index=True)
results_df.to_pickle('dataframe_recap_sampling.pkl')

In [6]:
results_df.describe(include='all')

,Model,Sampler,Sampling Strategy,F1 Score
count,32,32,32.0000,32.000000
unique,4,4,NaN,NaN
top,DecisionTreeClassifier,RandomOverSampler,NaN,NaN
freq,8,8,NaN,NaN
mean,NaN,NaN,0.0750,0.056036
std,NaN,NaN,0.0254,0.032969
min,NaN,NaN,0.0500,0.015576
25%,NaN,NaN,0.0500,0.031354
50%,NaN,NaN,0.0750,0.038366
75%,NaN,NaN,0.1000,0.072764


On va maintenant faire la même chose mais cette fois-ci sans les données rééchantilonnées

Modèle DecisionTreeClassifier:

In [ ]:
# On définit le modèle à tester
models = [DecisionTreeClassifier()]

# Initialisation d'un DataFrame pour stocker les résultats
results_df_no_sampling = pd.DataFrame(columns=['Model', 'F1 Score'])

# On parcourt chaque modèle
for model in models:
    # Entraîne le modèle sur les données d'entraînement d'origine
    model.fit(X_train, y_train)
            
    # Fait des prédictions sur l'ensemble de test
    y_pred = model.predict(X_test)

    # Évalue les performances du modèle
    f1 = f1_score(y_test, y_pred)

    # Ajoute les résultats au DataFrame
    results_df_no_sampling = pd.concat([results_df_no_sampling, pd.DataFrame({
        'Model': [model.__class__.__name__],
        'F1 Score': [f1]
    })], ignore_index=True)

    # Affiche le message de progression
    tqdm.write(f"Model: {model.__class__.__name__}, F1 Score: {f1}")


# Affiche le tableau récapitulatif
print(results_df_no_sampling)
results_df_no_sampling.to_pickle('dataframe_recap_no_sampling_1.pkl')

Modèle RandomForestClassifier:

In [ ]:
# On définit le modèle à tester
models = [RandomForestClassifier()]

# Initialisation d'un DataFrame pour stocker les résultats
results_df_no_sampling_2 = pd.DataFrame(columns=['Model', 'F1 Score'])

# On parcourt chaque modèle
for model in models:
    # Entraîne le modèle sur les données d'entraînement d'origine
    model.fit(X_train, y_train)
            
    # Fait des prédictions sur l'ensemble de test
    y_pred = model.predict(X_test)

    # Évalue les performances du modèle
    f1 = f1_score(y_test, y_pred)

    # Ajoute les résultats au DataFrame
    results_df_no_sampling_2 = pd.concat([results_df_no_sampling_2, pd.DataFrame({
        'Model': [model.__class__.__name__],
        'F1 Score': [f1]
    })], ignore_index=True)

    # Affiche le message de progression
    tqdm.write(f"Model: {model.__class__.__name__}, F1 Score: {f1}")

# Affiche le tableau récapitulatif
print(results_df_no_sampling_2)
results_df_no_sampling_2.to_pickle('dataframe_recap_no_sampling_2.pkl')

Modèle SVC:

In [ ]:
# On définit le modèle à tester
models = [SVC()]

# Initialisation d'un DataFrame pour stocker les résultats
results_df_no_sampling_3 = pd.DataFrame(columns=['Model', 'F1 Score'])

# On parcourt chaque modèle
for model in models:
    # Entraîne le modèle sur les données d'entraînement d'origine
    model.fit(X_train, y_train)
            
    # Fait des prédictions sur l'ensemble de test
    y_pred = model.predict(X_test)

    # Évalue les performances du modèle
    f1 = f1_score(y_test, y_pred)

    # Ajoute les résultats au DataFrame
    results_df_no_sampling_3 = pd.concat([results_df_no_sampling_3, pd.DataFrame({
        'Model': [model.__class__.__name__],
        'F1 Score': [f1]
    })], ignore_index=True)

    # Affiche le message de progression
    tqdm.write(f"Model: {model.__class__.__name__}, F1 Score: {f1}")

# Affiche le tableau récapitulatif
print(results_df_no_sampling_3)
results_df_no_sampling_3.to_pickle('dataframe_recap_no_sampling_3.pkl')

Modèle AdaBoostClassifier:

In [ ]:
# On définit le modèle à tester
models = [AdaBoostClassifier()]

# Initialisation d'un DataFrame pour stocker les résultats
results_df_no_sampling_4 = pd.DataFrame(columns=['Model', 'F1 Score'])

# On parcourt chaque modèle
for model in models:
    # Entraîne le modèle sur les données d'entraînement d'origine
    model.fit(X_train, y_train)
            
    # Fait des prédictions sur l'ensemble de test
    y_pred = model.predict(X_test)

    # Évalue les performances du modèle
    f1 = f1_score(y_test, y_pred)

    # Ajoute les résultats au DataFrame
    results_df_no_sampling_4 = pd.concat([results_df_no_sampling_4, pd.DataFrame({
        'Model': [model.__class__.__name__],
        'F1 Score': [f1]
    })], ignore_index=True)

    # Affiche le message de progression
    tqdm.write(f"Model: {model.__class__.__name__}, F1 Score: {f1}")

# Affiche le tableau récapitulatif
print(results_df_no_sampling_4)
results_df_no_sampling_4.to_pickle('dataframe_recap_no_sampling_4.pkl')

Modèle XGBClassifier:

In [ ]:
# On définit le modèle à tester
models = [XGBClassifier()]

# Initialisation d'un DataFrame pour stocker les résultats
results_df_no_sampling_5 = pd.DataFrame(columns=['Model', 'F1 Score'])

# On parcourt chaque modèle
for model in models:
    # Entraîne le modèle sur les données d'entraînement d'origine
    model.fit(X_train, y_train)
            
    # Fait des prédictions sur l'ensemble de test
    y_pred = model.predict(X_test)

    # Évalue les performances du modèle
    f1 = f1_score(y_test, y_pred)

    # Ajoute les résultats au DataFrame
    results_df_no_sampling_5 = pd.concat([results_df_no_sampling_5, pd.DataFrame({
        'Model': [model.__class__.__name__],
        'F1 Score': [f1]
    })], ignore_index=True)

    # Affiche le message de progression
    tqdm.write(f"Model: {model.__class__.__name__}, F1 Score: {f1}")

# Affiche le tableau récapitulatif
print(results_df_no_sampling_5)
results_df_no_sampling_5.to_pickle('dataframe_recap_no_sampling_5.pkl')

Modèle LogisticRegression:

In [ ]:
# On définit le modèle à tester
models = [LogisticRegression()]

# Initialisation d'un DataFrame pour stocker les résultats
results_df_no_sampling_6 = pd.DataFrame(columns=['Model','F1 Score'])

# On parcourt chaque modèle
for model in models:
    # Entraîne le modèle sur les données d'entraînement d'origine
    model.fit(X_train, y_train)
            
    # Fait des prédictions sur l'ensemble de test
    y_pred = model.predict(X_test)

    # Évalue les performances du modèle
    f1 = f1_score(y_test, y_pred)

    # Ajoute les résultats au DataFrame
    results_df_no_sampling_6 = pd.concat([results_df_no_sampling_6, pd.DataFrame({
        'Model': [model.__class__.__name__],
        'F1 Score': [f1]
    })], ignore_index=True)

    # Affiche le message de progression
    tqdm.write(f"Model: {model.__class__.__name__},  F1 Score: {f1}")

# Affiche le tableau récapitulatif
print(results_df_no_sampling_6)
results_df_no_sampling_6.to_pickle('dataframe_recap_no_sampling_6.pkl')

On réunit les résultats sans rééchantillonage dans un seul tableau récapitulatif

In [ ]:
results_df_no_sampling_1 = pd.read_pickle('dataframe_recap_no_sampling.pkl')
results_df_no_sampling_2 = pd.read_pickle('dataframe_recap_no_sampling_2.pkl')
results_df_no_sampling_4 = pd.read_pickle('dataframe_recap_no_sampling_4.pkl')
results_df_no_sampling_5 = pd.read_pickle('dataframe_recap_no_sampling_5.pkl')
results_df_no_sampling_6 = pd.read_pickle('dataframe_recap_no_sampling_6.pkl')

In [ ]:
results_df_no_sampling = pd.concat([results_df_no_sampling_1, results_df_no_sampling_2, results_df_no_sampling_4, results_df_no_sampling_5, results_df_no_sampling_6], ignore_index=True)
results_df_no_sampling.to_pickle('dataframe_recap_no_sampling.pkl')

In [ ]:
results_df_no_sampling.describe(include='all')

On rassemble nos deux tableaux récapitulatifs pour avoir tous nos résultats dans un seul tableau

In [ ]:
result_sampling = pd.read_pickle('dataframe_recap_sampling1.pkl')
result_no_sampling = pd.read_pickle('dataframe_recap_no_sampling.pkl')

In [ ]:
results_df = pd.concat([result_sampling, result_no_sampling])
results_df.to_pickle('dataframe_recap_sampling_no_sampling.pkl')

In [ ]:
results_df.reset_index(drop=True, inplace=True)

In [ ]:
results_df.describe(include='all')

Avec notre tableau récapitulatif global, nous allons maintenant déterminer les meilleurs modèles:

In [7]:
# On groupe les résultats par modèle
grouped_results = results_df.groupby('Model')

# Initialisation d'un DataFrame pour stocker les meilleurs résultats par modèle
best_results_df = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# On parcourt chaque groupe
for model, group_df in grouped_results:
    # Trouve la ligne avec le plus grand F1 score pour chaque modèle
    best_result = group_df.sort_values(by='F1 Score', ascending=False).iloc[0]
    
    # Ajoute le meilleur résultat au DataFrame final
    # Remplace la ligne best_results_df = best_results_df.append(...) par ceci
    best_results_df = pd.concat([best_results_df, pd.DataFrame({
        'Model': [best_result['Model']],
        'Sampler': [best_result['Sampler']],
        'Sampling Strategy': [best_result['Sampling Strategy']],
        'F1 Score': [best_result['F1 Score']]
    })], ignore_index=True)

# Affiche le tableau récapitulatif des meilleurs résultats pour chaque modèle
print(best_results_df)
best_results_df.to_pickle('best_results_df.pkl')

                    Model             Sampler  Sampling Strategy  F1 Score
0      AdaBoostClassifier   RandomOverSampler               0.10  0.131428
1  DecisionTreeClassifier               SMOTE               0.05  0.037149
2  RandomForestClassifier  RandomUnderSampler               0.05  0.102525
3           XGBClassifier               SMOTE               0.10  0.077850


C:\Users\marevel\AppData\Local\Temp\ipykernel_16984\346220120.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  best_results_df = pd.concat([best_results_df, pd.DataFrame({


In [8]:
best_results_df

,Model,Sampler,Sampling Strategy,F1 Score
0,AdaBoostClassifier,RandomOverSampler,0.10,0.131428
1,DecisionTreeClassifier,SMOTE,0.05,0.037149
2,RandomForestClassifier,RandomUnderSampler,0.05,0.102525
3,XGBClassifier,SMOTE,0.10,0.077850


## Maintenant qu'on a une vue d'ensemble claire sur les méthodes de re-échantillonage qui marchent le mieux en fonction des modèles, on va reprendre ces combinaisons pour réaliser une optimisation cette fois-ci des hyper-paramètres des modèles à l'aide d'un greedsearch.

Modèle AdaBoostClassifier:

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
import pickle
from imblearn.over_sampling import RandomOverSampler
from tqdm import tqdm

# Sous-échantillonnage
ros = RandomOverSampler(sampling_strategy=0.10)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

# Modèle de l'AdaBoostClassifier
adab_classifier = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())  # Utilise un arbre de décision faible par défaut

# Pipeline
pipeline = Pipeline([
    ('classifier', adab_classifier)
])

# Crée un objet TimeSeriesSplit pour la cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Paramètres pour la recherche sur la grille
param_grid = {
    'classifier__n_estimators': [50, 100],
    'classifier__base_estimator__max_depth': [2, 3],  # Ajustez ici pour le paramètre de l'arbre faible
    'classifier__learning_rate': [0.1, 0.5],
}

# Recherche sur la grille
grid_search = GridSearchCV(pipeline, param_grid, scoring='f1', cv=tscv)

# Barre de progression pour grid search
for train_index, test_index in tqdm(tscv.split(X_train, y_train), desc="Grid Search Progress"):
    X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]

    grid_search.fit(X_train_fold, y_train_fold)

# Affiche les meilleurs paramètres et le meilleur score F1
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score F1:", grid_search.best_score_)

# Enregistrement du modèle
with open('modele_adaboost.pkl', 'wb') as file:
    pickle.dump(grid_search, file)


Modèle RandomForestClassifier:

In [17]:
from sklearn.ensemble import RandomForestClassifier

# Sous-échantillonnage
rus = RandomUnderSampler(sampling_strategy=0.05)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

# Modèle du RandomForestClassifier
rf_classifier = RandomForestClassifier()

# Pipeline
pipeline_rf = Pipeline([
    ('classifier', rf_classifier)
])

tscv = TimeSeriesSplit(n_splits=5)

# Paramètres pour la recherche sur la grille
param_grid_rf = {
    'classifier__n_estimators': [50, 100],
    'classifier__max_depth': [None, 10, 20],  # Ajustez ici pour le paramètre de profondeur
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
}

# Recherche sur la grille
grid_search_rf = GridSearchCV(pipeline_rf, param_grid_rf, scoring='f1', cv=tscv)
grid_search_rf.fit(X_train, y_train)

# Affiche les meilleurs paramètres et le meilleur score F1
print("Meilleurs paramètres (Random Forest):", grid_search_rf.best_params_)
print("Meilleur score F1 (Random Forest):", grid_search_rf.best_score_)

with open('modele_random_forest.pkl', 'wb') as file_rf:
    pickle.dump(grid_search_rf, file_rf)


Modèle XGBClassifier:

In [16]:
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
import pickle

# Sous-échantillonnage
ros = RandomOverSampler(sampling_strategy=0.10)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

# Modèle du XGBClassifier
xgb_classifier = XGBClassifier()

# Pipeline
pipeline_xgb = Pipeline([
    ('classifier', xgb_classifier)
])

tscv = TimeSeriesSplit(n_splits=5)

# Paramètres pour la recherche sur la grille
param_grid_xgb = {
    'classifier__n_estimators': [50, 100],
    'classifier__max_depth': [3, 7],  # Ajustez ici pour le paramètre de profondeur
    'classifier__learning_rate': [0.1, 0.01],
    'classifier__subsample': [0.8, 1.0],
    'classifier__colsample_bytree': [0.8, 1.0],
}

# Recherche sur la grille
grid_search_xgb = GridSearchCV(pipeline_xgb, param_grid_xgb, scoring='f1', cv=tscv)
grid_search_xgb.fit(X_train, y_train)

# Affiche les meilleurs paramètres et le meilleur score F1
print("Meilleurs paramètres (XGBoost):", grid_search_xgb.best_params_)
print("Meilleur score F1 (XGBoost):", grid_search_xgb.best_score_)

# Affiche les résultats détaillés de la recherche sur la grille
results = grid_search_xgb.cv_results_
for mean_score, params in zip(results["mean_test_score"], results["params"]):
    print(f"Score moyen: {mean_score}, Paramètres: {params}")

# Enregistrement du modèle
with open('modele_xgboost.pkl', 'wb') as file_xgb:
    pickle.dump(grid_search_xgb, file_xgb)


Meilleurs paramètres (XGBoost): {'classifier__colsample_bytree': 0.8, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 7, 'classifier__n_estimators': 100, 'classifier__subsample': 0.8}
Meilleur score F1 (XGBoost): 0.09887753490587534
Score moyen: 0.029262497240361184, Paramètres: {'classifier__colsample_bytree': 0.8, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__n_estimators': 50, 'classifier__subsample': 0.8}
Score moyen: 0.028306750383856456, Paramètres: {'classifier__colsample_bytree': 0.8, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__n_estimators': 50, 'classifier__subsample': 1.0}
Score moyen: 0.05709189714118945, Paramètres: {'classifier__colsample_bytree': 0.8, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__n_estimators': 100, 'classifier__subsample': 0.8}
Score moyen: 0.05738477767642375, Paramètres: {'classifier__colsample_bytree': 0.8, 'classifier__learning_rate': 0.1, 'classifier

Modèle GradientBoostingClassifier:

In [35]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline

# Sous-échantillonnage
rus = RandomUnderSampler(sampling_strategy=0.1)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

# Modèle de l'arbre de décision
gb_classifier = GradientBoostingClassifier()

# Pipeline
pipeline = Pipeline([
    ('classifier', gb_classifier)
])

# Crée un objet TimeSeriesSplit pour la cross-validation
tscv = TimeSeriesSplit(n_splits = 5)

# Paramètres pour la recherche sur la grille
param_grid = {
    'classifier__n_estimators': [50, 100],
    'classifier__max_depth': [10, 15],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [2, 4]
}

# Recherche sur la grille
grid_search = GridSearchCV(pipeline, param_grid, scoring='f1', cv=tscv)
grid_search.fit(X_train, y_train)

 
# Affiche les meilleurs paramètres et le meilleur score F1
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score F1:", grid_search.best_score_)

# Enregistrement du modèle
with open('modele_gboost.pkl', 'wb') as file_gb:
    pickle.dump(gb_classifier, file_gb)

Meilleurs paramètres: {'classifier__max_depth': 15, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 100}
Meilleur score F1: 0.10323029387708163


Modèle GradientBoostingClassifier:

In [17]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline

# Sous-échantillonnage
ros = RandomOverSampler(sampling_strategy=0.1)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

# Modèle de l'arbre de décision
gb_classifier = GradientBoostingClassifier()

# Pipeline
pipeline = Pipeline([
    ('classifier', gb_classifier)
])

# Crée un objet TimeSeriesSplit pour la cross-validation
tscv = TimeSeriesSplit(n_splits = 5)

# Paramètres pour la recherche sur la grille
param_grid = {
    'classifier__n_estimators': [50, 100],
    'classifier__max_depth': [5, 10, 15],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [2, 4]
}

# Recherche sur la grille
grid_search = GridSearchCV(pipeline, param_grid, scoring='f1', cv=tscv)
grid_search.fit(X_train_ros, y_train_ros)

 
# Affiche les meilleurs paramètres et le meilleur score F1
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score F1:", grid_search.best_score_)

In [44]:
import numpy as np

# Compte le nombre d'éléments uniques dans y_train_rus
unique_classes, counts = np.unique(y_train_rus, return_counts=True)

# Affiche le nombre d'éléments uniques et leurs occurrences
for cls, count in zip(unique_classes, counts):
    print(f"Classe {cls}: {count} occurrences")

Classe 0: 2290 occurrences
Classe 1: 229 occurrences


In [33]:
len(y_train)

38827

In [34]:
len(y_train_rus)

2519

In [30]:
# Utilise le meilleur modèle trouvé par la recherche sur la grille
best_model = grid_search.best_estimator_

# Fait des prédictions sur l'ensemble de test
y_pred = best_model.predict(X_test)

# Calcule le F1-score sur l'ensemble de test
f1_test = f1_score(y_test, y_pred)

# Affichage du F1-score sur l'ensemble de test
print("F1 Score sur l'ensemble de test:", f1_test)

F1 Score sur l'ensemble de test: 0.054590570719602965
